In [1]:
%pylab inline
import pandas as pd
from glob2 import glob
from pathlib import Path
from sqlalchemy import create_engine
from common.utils import init_cached_database, parse_mongodb_connection_string
from src.utils import get_config

Populating the interactive namespace from numpy and matplotlib


In [2]:
config = get_config('brca-database')

2021-07-25 10:31:58.388 | DEBUG    | src.utils:get_config:10 - Reading brca-database from /home/ohad/Projects/ohad/src/../config.toml


In [3]:
xlsx_files = glob(
    "/data/Database/Human/TCGA-BRCA_Online/Clinical Data (CSV)/*.xlsx")


In [5]:
db = init_cached_database(parse_mongodb_connection_string(**config), db_name=config['db_name'])

In [17]:
def split_name(name):
    return Path(name).name.split('org_')[-1].split('.xlsx')[0]

def replace_not_available_with_none(item):
    for key, value in item.items():
        if item[key] in ['[Not Available]', '[Not Applicable]']:
            item[key] = None
    return item


In [20]:
for xlsx_file in xlsx_files:
    table_name = split_name(xlsx_file)
    df = pd.read_excel(xlsx_file,engine='openpyxl')[2:]
    # df.to_sql(table_name, con=engine, if_exists='replace')
    items = [row.to_dict() for _, row in df.iterrows()]
    items = [replace_not_available_with_none(item) for item in items]
    db[table_name].insert_many(items)



In [19]:
items[0]

{'bcr_patient_uuid': 'e17c565c-9857-4df3-a352-903b53093c85',
 'bcr_patient_barcode': 'TCGA-AO-A0JI',
 'bcr_omf_barcode': 'TCGA-AO-A0JI-O7952',
 'bcr_omf_uuid': '2EB50E2A-F7D5-4CF1-B85D-2935532A495B',
 'form_completion_date': datetime.datetime(2011, 2, 18, 0, 0),
 'malignancy_type': 'Prior Malignancy',
 'other_malignancy_dx_days_to': None,
 'surgery_indicator': None,
 'other_malignancy_surgery_type': None,
 'other_malignancy_surgery_days_to': -731,
 'pharmaceutical_therapy_indicator': 'NO',
 'pharmaceutical_therapy_extent': None,
 'pharmaceutical_therapy_drug_name': None,
 'pharmaceutical_tx_started_days_to': None,
 'radiation_therapy_indicator': 'NO',
 'radiation_therapy_extent': None,
 'history_rt_tx_to_site_of_tcga_tumor': None,
 'radiation_therapy_started_days_to': None,
 'ajcc_staging_edition': None,
 'ajcc_tumor_pathologic_pt': None,
 'ajcc_nodes_pathologic_pn': None,
 'ajcc_metastasis_pathologic_pm': None,
 'ajcc_pathologic_tumor_stage': 'Stage IA',
 'clinical_stage': None,
 'oth

In [47]:
patients = pd.concat([pd.read_excel(xlsx_file, engine='openpyxl')[2:] for xlsx_file in xlsx_files])
patients = pd.concat((patients['bcr_patient_uuid'],
                     patients['bcr_patient_barcode']), axis=1).drop_duplicates()

    


In [49]:
patients

,bcr_patient_uuid,bcr_patient_barcode
2,095c7985-3842-494f-b591-706ad1cd2133,TCGA-AO-A03M
3,a6cb373a-1604-4e25-bd81-da6c3e4f06ee,TCGA-AO-A03V
4,02bf5203-f9cd-4c5a-97b4-e5584dc22325,TCGA-AO-A0J8
5,a4903de8-6cf5-4541-8ec7-065beace8b44,TCGA-AO-A0J9
6,3ea9412b-429e-4dcf-90cd-c53be06885a1,TCGA-AO-A0JB
...,...,...
110,5ff12cbe-a3e0-4ce6-8452-12b984a1baaf,TCGA-EW-A1IY
112,a381275f-8858-4793-a8a2-0ca9db7274c8,TCGA-EW-A1J2
113,87281a89-91d2-44f7-9f80-668567ad5c72,TCGA-EW-A1J6
115,8f912a40-3251-4e1f-ae65-1f93d71f9881,TCGA-EW-A1OZ


In [51]:
# patients.to_sql('patients', con=engine, if_exists='replace')


In [54]:
len('TCGA-AO-A03M')


12